#Купил значит я себе рахат-лукум, как в Греции

In [ ]:
!pip install catboost # Теперь я вич инфецированный
!pip install unrar

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import numpy as np
import pandas as pd
import math
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [1]:
!unzip /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip

unzip:  cannot find or open /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip, /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip.zip or /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip.ZIP.


In [2]:
# Функция для расчета расстояния между координатами
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в километрах
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    return R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

In [3]:
# Загрузка данных
data = pd.read_csv('new-york-city-taxi-fare-prediction/train.csv', nrows=1000000)
data = data.dropna() # Убираем пустые значения

In [4]:
# Преобразование даты и времени
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], errors='coerce')
data = data.dropna(subset=['pickup_datetime'])

In [5]:
# Создание временных признаков
data['year'] = data['pickup_datetime'].dt.year
data['month'] = data['pickup_datetime'].dt.month
data['day'] = data['pickup_datetime'].dt.day
data['day_of_week'] = data['pickup_datetime'].dt.dayofweek
data['hour'] = data['pickup_datetime'].dt.hour

In [6]:
# Создание признака расстояния
data['distance'] = haversine_distance(data['pickup_latitude'], data['pickup_longitude'],
                                      data['dropoff_latitude'], data['dropoff_longitude'])

In [7]:
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,day_of_week,hour,distance
count,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000,999990.000000
mean,11.347953,-72.526699,39.929040,-72.527860,39.919954,1.684941,2011.741103,6.267841,15.697996,3.039870,13.509463,19.090118
std,9.821790,12.057778,7.626087,11.324494,8.201418,1.323907,1.860761,3.436230,8.682472,1.949966,6.513855,362.408033
min,-44.900000,-3377.680935,-3116.285383,-3383.296608,-3114.338567,0.000000,2009.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,6.000000,-73.992060,40.734965,-73.991385,40.734046,1.000000,2010.000000,3.000000,8.000000,1.000000,9.000000,1.212942
50%,8.500000,-73.981792,40.752695,-73.980135,40.753166,1.000000,2012.000000,6.000000,16.000000,3.000000,14.000000,2.115922
75%,12.500000,-73.967094,40.767154,-73.963654,40.768129,2.000000,2013.000000,9.000000,23.000000,5.000000,19.000000,3.882433
max,500.000000,2522.271325,2621.628430,45.581619,1651.553433,208.000000,2015.000000,12.000000,31.000000,6.000000,23.000000,16878.303157


In [8]:
#Загрузка данных ключевых точек
key_points = pd.read_csv('interes_points.csv')
key_points

,type,name,latitude,longitude
0,Airport,Guardia,40.775235,-73.874517
1,Airport,John F. Kennedy International Airport,40.642100,-73.789290
2,Street,Wall Street,40.706974,-74.009697
3,Park,Manhattan,40.778226,-73.968323
4,Railway station,Grand Central Station,40.752631,-73.977364
5,Street,Times Square,40.757566,-73.985786
6,Street,Brodway,40.794742,-73.971547
7,Building,Empire State Building,40.748427,-73.985563
8,Park,National September 11th Memorial,40.711629,-74.013478
9,Building,Brooklyn Bridge,40.704352,-73.994877


In [ ]:
# Создание признака раccтояния до ключевых точек

pn = key_points[['latitude', 'longitude']].values
for i in range(len(pn)):

    elem = pn[i]
    data[f'distance_{i}'] = haversine_distance(data['pickup_longitude'], data['pickup_latitude'], elem[0], elem[1])


In [20]:
data

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,...,distance_0,distance_1,distance_2,distance_3,distance_4,distance_5,distance_6,distance_7,distance_8,distance_9
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,...,15082.865136,15066.465383,15079.906691,15086.033298,15083.655389,15084.423542,15087.843566,15083.469313,15080.504487,15079.183459
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,...,15079.229220,15062.835267,15076.214597,15082.366870,15079.981662,15080.747861,15084.178853,15079.792168,15076.811925,15075.495826
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,...,15081.388879,15064.993861,15078.384361,15084.532025,15082.148128,15082.914679,15086.343702,15081.959248,15078.981774,15077.664787
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,...,15080.453532,15064.058617,15077.448032,15083.596144,15081.212119,15081.978636,15085.407851,15081.023179,15078.045437,15076.728536
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,...,15081.879242,15065.483732,15078.879415,15085.024939,15082.641650,15083.408364,15086.836473,15082.453055,15079.476868,15078.159468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2014-09-13 21:44:38.0000001,7.0,2014-09-13 21:44:38+00:00,-73.976676,40.785630,-73.959196,40.775114,1,2014,9,...,15082.243579,15065.848393,15079.240650,15085.387590,15083.003898,15083.770505,15087.199219,15082.815115,15079.838076,15078.520949
999996,2010-09-20 14:50:37.0000002,7.3,2010-09-20 14:50:37+00:00,-73.992103,40.671385,-73.992103,40.671385,2,2010,9,...,15078.491881,15062.097033,15075.485754,15081.634150,15079.250044,15080.016539,15083.445876,15079.061066,15076.083154,15074.766307
999997,2013-04-26 14:03:00.000000118,10.5,2013-04-26 14:03:00+00:00,-73.978423,40.751135,-73.993397,40.762197,5,2013,4,...,15081.167216,15064.772032,15078.164278,15084.311221,15081.927528,15082.694135,15086.122850,15081.738745,15078.761705,15077.444578
999998,2011-07-08 00:29:00.00000099,6.9,2011-07-08 00:29:00+00:00,-73.980317,40.759482,-73.976832,40.743122,1,2011,7,...,15081.382296,15064.987191,15078.378601,15084.525890,15082.142099,15082.908679,15086.337542,15081.953269,15078.976021,15077.658961


In [ ]:
features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
            'passenger_count', 'year', 'month', 'day', 'day_of_week', 'hour', 'distance', 'distance_0', 'distance_1', 'distance_2'] #, 'distance_3', 'distance_4', 'distance_5', 'distance_6', 'distance_7', 'distance_8', 'distance_9']
target = 'fare_amount'
X = data[features]
y = data[target]

# Разделение данных
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# Создание и обучение модели
model = CatBoostRegressor(iterations=1200, learning_rate=0.06, depth=10, l2_leaf_reg = 5, loss_function='RMSE', verbose=100, random_seed=52)
model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)

0:	learn: 9.3985958	test: 9.4183645	best: 9.4183645 (0)	total: 50.9ms	remaining: 1m 41s
100:	learn: 4.3739076	test: 4.4883848	best: 4.4883848 (100)	total: 5.37s	remaining: 1m 41s
200:	learn: 4.1511928	test: 4.3779340	best: 4.3779340 (200)	total: 10.7s	remaining: 1m 35s
300:	learn: 4.0174175	test: 4.3417290	best: 4.3417290 (300)	total: 16s	remaining: 1m 30s
400:	learn: 3.9212258	test: 4.3146528	best: 4.3146528 (400)	total: 21.5s	remaining: 1m 25s
500:	learn: 3.8441015	test: 4.2987483	best: 4.2987180 (498)	total: 26.7s	remaining: 1m 20s
600:	learn: 3.7796901	test: 4.2876556	best: 4.2876556 (600)	total: 32.1s	remaining: 1m 14s
700:	learn: 3.7234753	test: 4.2808920	best: 4.2808920 (700)	total: 37.4s	remaining: 1m 9s
800:	learn: 3.6723425	test: 4.2744437	best: 4.2742471 (798)	total: 42.8s	remaining: 1m 4s
900:	learn: 3.6282315	test: 4.2710245	best: 4.2709549 (898)	total: 48.1s	remaining: 58.6s
1000:	learn: 3.5886708	test: 4.2672751	best: 4.2670112 (998)	total: 53.3s	remaining: 53.2s
1100:	l

In [33]:
# Оценка модели
y_pred = model.predict(X_val)
rmse = math.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE на тестовой выборке: {rmse:.2f}")

RMSE на тестовой выборке: 4.26


In [34]:
# Загрузка тестовых данных
test_data = pd.read_csv('new-york-city-taxi-fare-prediction/test.csv')

# Предобработка тестовых данных
test_data['pickup_datetime'] = pd.to_datetime(test_data['pickup_datetime'], errors='coerce')
test_data['year'] = test_data['pickup_datetime'].dt.year
test_data['month'] = test_data['pickup_datetime'].dt.month
test_data['day'] = test_data['pickup_datetime'].dt.day
test_data['day_of_week'] = test_data['pickup_datetime'].dt.dayofweek
test_data['hour'] = test_data['pickup_datetime'].dt.hour

test_data['distance'] = haversine_distance(test_data['pickup_latitude'], test_data['pickup_longitude'],
                                      test_data['dropoff_latitude'], test_data['dropoff_longitude'])

In [35]:
# Создание признака раccтояния до ключевых точек

pn = key_points[['latitude', 'longitude']].values
for i in range(len(pn)):

    elem = pn[i]
    test_data[f'distance_{i}'] = haversine_distance(test_data['pickup_longitude'], test_data['pickup_latitude'], elem[0], elem[1])


In [36]:
predictions = model.predict(test_data[features]) # ТО САМОЕ ПРЕДСКАЗАНИЕ

# Создаем DataFrame для отправки
submission = pd.DataFrame({
    'key': test_data['key'],
    'fare_amount': predictions
    })

# Сохранение результатов
submission.to_csv('taxi_fare_submission.csv', index=False)
print("Файл taxi_fare_submission.csv успешно создан.")

Файл taxi_fare_submission.csv успешно создан.
